# Exercise 11

## Car Price Prediction

Predict if the price of a car is low or high

In [1]:
%matplotlib inline
import pandas as pd

data = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTrain_carListings.zip')
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
15,2016,29242,0,0,0,0,1,0,0,1
47,2015,26465,0,0,0,0,1,0,0,1
85,2012,46739,0,1,0,0,0,0,0,1
141,2017,41722,0,0,0,0,0,1,0,1
226,2014,77669,0,0,0,0,0,0,1,0


In [2]:
data.shape

(13150, 10)

In [3]:
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# Exercise 11.1

Estimate a Decision Tree Classifier Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [9]:
import pandas as pd
import numpy as np
max_depth = None
num_pct = 10
max_features = None
min_gain=0.001
j = 1
print(X.columns[j])

Mileage


In [10]:
# Split the variable in num_ctp points
splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / num_pct).tolist())
splits = np.unique(splits)

In [12]:
k=5
filter_l = X.iloc[:, j] < splits[k]

y_l = y.loc[filter_l]
y_r = y.loc[~filter_l]

In [15]:
# definición de la función Gini
def gini(y):
    if y.shape[0] == 0:
        return 0
    else:
        return 1 - (y.mean()**2 + (1 - y.mean())**2)
gini_l = gini(y_l)
gini_r = gini(y_r)

In [16]:
#Definición de la función que calcula el nivel de impureza de Gini

def gini_impurity(X_col, y, split):
    "Calculate the gain of an split k on feature j"
    
    filter_l = X_col < split
    y_l = y.loc[filter_l]
    y_r = y.loc[~filter_l]
    
    n_l = y_l.shape[0]
    n_r = y_r.shape[0]
    
    gini_y = gini(y)
    gini_l = gini(y_l)
    gini_r = gini(y_r)
    
    gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r)
    
    return gini_impurity_
gini_impurity(X.iloc[:, j], y, splits[k])

0.18496148274516044

In [17]:
#Probando todas las particiones posibles
def best_split(X, y, num_pct=10):
    
    features = range(X.shape[1])
    
    best_split = [0, 0, 0]  # j, split, gain
    
    # For all features
    for j in features:
        
        splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (num_pct+1)).tolist())
        splits = np.unique(splits)[1:]
        
        # For all splits
        for split in splits:
            gain = gini_impurity(X.iloc[:, j], y, split)
                        
            if gain > best_split[2]:
                best_split = [j, split, gain]
    
    return best_split

In [19]:
j, split, gain = best_split(X, y, 5)
j, split, gain
filter_l = X.iloc[:, j] < split
y_l = y.loc[filter_l]
y_r = y.loc[~filter_l]
y.shape[0], y_l.shape[0], y_r.shape[0]
y.mean(), y_l.mean(), y_r.mean()

(0.5795437262357415, 0.07939553849844087, 0.8117136176372342)

In [32]:
def tree_grow(X_train, y, level=0, min_gain=0.001, max_depth=None, num_pct=10):
    
    # If only one observation
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0)
        return tree
    
    # Calculate the best split
    j, split, gain = best_split(X, y, num_pct)
    
    # save tree and estimate prediction
    y_pred = int(y.mean() >= 0.5) 
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Laplace correction
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=gain)
    
    # Check stooping criteria
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # No stooping criteria was meet, then continue to create the partition
    filter_l = X.iloc[:, j] < split
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]
    tree['gain'] = gain
    # Next iteration to each split
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    tree['sr'] = tree_grow(X_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    
    return tree

In [33]:
tree_grow(X_train, y_train, level=0, min_gain=0.001, max_depth=3, num_pct=10)

{'y_pred': 1,
 'y_prob': 0.5780753517930095,
 'level': 0,
 'split': [1, 51704.54545454545],
 'n_samples': 13150,
 'gain': 0.23921463920774305,
 'sl': {'y_pred': 1,
  'y_prob': 0.8410963812253672,
  'level': 1,
  'split': [0, 2014.0],
  'n_samples': 13150,
  'gain': 0.03239749424488661,
  'sl': {'y_pred': 0,
   'y_prob': 0.3723958333333333,
   'level': 2,
   'split': [0, 2012.0],
   'n_samples': 13150,
   'gain': 0.05994163652516632,
   'sl': {'y_pred': 0,
    'y_prob': 0.08163265306122448,
    'level': 3,
    'split': -1,
    'n_samples': 13150,
    'gain': 0.007739335317460239},
   'sr': {'y_pred': 0,
    'y_prob': 0.4722222222222222,
    'level': 3,
    'split': -1,
    'n_samples': 13150,
    'gain': 0.03309574304406826}},
  'sr': {'y_pred': 1,
   'y_prob': 0.875576923076923,
   'level': 2,
   'split': [0, 2015.0],
   'n_samples': 13150,
   'gain': 0.01476433084662182,
   'sl': {'y_pred': 1,
    'y_prob': 0.7366597366597366,
    'level': 3,
    'split': -1,
    'n_samples': 13150,
 

In [34]:
tree = tree_grow(X_train, y_train, level=0, min_gain=0.001, max_depth=3, num_pct=10)

In [35]:
#Predicción del modelo
def tree_predict(X, tree, proba=False):
    
    predicted = np.ones(X.shape[0])

    # Check if final node
    if tree['split'] == -1:
        if not proba:
            predicted = predicted * tree['y_pred']
        else:
            predicted = predicted * tree['y_prob']
            
    else:
        
        j, split = tree['split']
        filter_l = (X.iloc[:, j] < split)
        X_l = X.loc[filter_l]
        X_r = X.loc[~filter_l]

        if X_l.shape[0] == 0:  # If left node is empty only continue with right
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)
        elif X_r.shape[0] == 0:  # If right node is empty only continue with left
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
        else:
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)

    return predicted 

In [37]:
tree_predict(X_train, tree).shape

(8810,)

In [46]:
#construcción tree predicción
y_pred_test=tree_predict(X_test, tree)

In [47]:
y_pred_test

array([0., 1., 1., ..., 1., 1., 0.])

In [48]:
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_test,y_pred_test))

Accuracy:0.868433


# Exercise 11.2

Estimate a Bagging of 10 Decision Tree Classifiers Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [154]:
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 10

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples

[array([3582, 1346, 5218, ..., 5288, 5094, 8043]),
 array([ 431, 3426, 8463, ..., 8218,  237, 8326]),
 array([8477, 4773, 5743, ..., 1809,  214, 6910]),
 array([5885, 1492, 7249, ..., 1834,  137, 7586]),
 array([ 352, 4574, 5708, ..., 7142, 2081, 7438]),
 array([2311, 6342, 4124, ..., 8286, 8453, 1222]),
 array([5003, 7878, 7832, ..., 3522,  407, 3676]),
 array([6901, 1829, 1437, ..., 8100, 3585,  522]),
 array([2874, 2197, 2888, ...,  477, 5922, 4068]),
 array([5547, 7784, 7473, ..., 4908, 6648, 3622])]

In [155]:
y_pred = pd.DataFrame(index=X_test.index, columns=[list(range(n_B))])

In [156]:
# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X_train_b = X_train.iloc[sample, 0:]
    y_train_b = y_train.iloc[samples[0]]
    tree = tree_grow(X_train_b, y_train_b, level=0, min_gain=0.001, max_depth=3, num_pct=10)
    y_pred[i] = tree_predict(X_test, tree)

In [157]:
tree

{'y_pred': 1,
 'y_prob': 0.5795506128007263,
 'level': 0,
 'split': [1, 51704.54545454545],
 'n_samples': 13150,
 'gain': 0.23995065305181007,
 'sl': {'y_pred': 1,
  'y_prob': 0.8411764705882353,
  'level': 1,
  'split': [0, 2015.0],
  'n_samples': 13150,
  'gain': 0.03501307958501437,
  'sl': {'y_pred': 1,
   'y_prob': 0.6542155816435432,
   'level': 2,
   'split': [0, 2014.0],
   'n_samples': 13150,
   'gain': 0.043142883885472605,
   'sl': {'y_pred': 0,
    'y_prob': 0.36828644501278773,
    'level': 3,
    'split': -1,
    'n_samples': 13150,
    'gain': 0.04999420210321004},
   'sr': {'y_pred': 1,
    'y_prob': 0.7292929292929293,
    'level': 3,
    'split': -1,
    'n_samples': 13150,
    'gain': 0.024756713975353672}},
  'sr': {'y_pred': 1,
   'y_prob': 0.9347244515783841,
   'level': 2,
   'split': [1, 33597.909090909096],
   'n_samples': 13150,
   'gain': 0.00580302892324655,
   'sl': {'y_pred': 1,
    'y_prob': 0.9776315789473684,
    'level': 3,
    'split': -1,
    'n_samp

In [158]:
y_pred

,0,1,2,3,4,5,6,7,8,9
332784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146436,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
130476,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
85618,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
330419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205915,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2836,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
250833,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
126784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
y_pred_b = ((y_pred.sum(axis=1)[:])/n_B)

In [160]:
#Usando el criterio de majority voting

y_pred_b

332784    0.0
146436    1.0
130476    1.0
85618     1.0
75474     0.0
330419    0.0
205915    1.0
2836      1.0
250833    1.0
126784    0.0
70347     1.0
63289     1.0
112205    1.0
110630    1.0
458256    1.0
371230    0.0
130467    0.0
457797    1.0
367064    0.0
131705    1.0
484284    0.0
482990    0.0
294535    0.0
365460    0.0
64466     1.0
356183    0.0
332219    1.0
480099    1.0
250250    1.0
116789    1.0
         ... 
320236    0.0
105104    1.0
214655    0.0
266700    0.0
115954    0.0
273953    0.0
334056    1.0
417586    0.0
74732     1.0
118259    0.0
431992    0.0
475814    1.0
43386     0.0
452397    1.0
392445    1.0
246910    0.0
323580    1.0
192225    1.0
252987    1.0
178362    1.0
58768     1.0
123468    0.0
272289    1.0
325203    1.0
86246     1.0
244795    1.0
366403    0.0
269482    1.0
5010      1.0
310628    0.0
Length: 4340, dtype: float64

In [161]:
from sklearn.metrics import accuracy_score
print('Accuracy:%2f'% accuracy_score(y_test,y_pred_b))

Accuracy:0.865899


# Exercise 11.3

Implement the variable max_features on the Decision Tree Classifier created in 11.1.

Compare the impact in the results by varing the parameter max_features

Evaluate the accuracy on the testing set

# Exercise 11.4

Estimate a Bagging of 10 Decision Tree Classifiers with `max_features = log(n_features)`

Evaluate the accuracy on the testing set

# Exercise 11.5

Using sklearn, train a RandomForestClassifier

Evaluate the accuracy on the testing set

# Exercise 11.6

Find the best parameters of the RandomForestClassifier (max_depth, max_features, n_estimators)

Evaluate the accuracy on the testing set